In [22]:
%matplotlib inline
import os
import glob
import doctest

import importlib
from src import ParticleFrames
import src.ParticleFrames as pframes
importlib.reload(pframes)

ParticleFrame = pframes.ParticleFrame
EnsembleFrames = pframes.EnsembleFrames

mid_sorted = EnsembleFrames.mid_sorted
tail_sorted = EnsembleFrames.tail_sorted

In [23]:
doctest.testmod(ParticleFrames)

TestResults(failed=0, attempted=19)

In [24]:
tpdir = '../test/tdm_iso/datapr/ensemble/'
tpfs = sorted(glob.glob(tpdir+'tdm_eta5_ensemble*'),key=tail_sorted)
des = glob.glob(tpdir+'*eta5*destroy*')[0]
ens = EnsembleFrames(tpfs)
desf = ParticleFrame(des)

In [25]:
f1 = lambda df: df.get_indexed(desf.get_erange(1,1000)['index'])
ens.plot_yproj(interval=100, hdf=f1)

In [28]:
singlef = ens.derive_single_traj(574467)
singlef.plot_traj(bin=20)

dek1: 382.8516788794164
dek2: -8.924759623529628
dek3: -2.3934717688279132
